# Model Selection
### `! git clone https://github.com/ds4e/model_selection`

## Introduction
- Why and how do we build models?
- There are three key pieces:
    1. The variables in the raw data
    2. Cleaned variables and transformations of them (the feature space) --> need to do feature modeling make model usable
    3. The model class (hyperparameter choices: variable selection, "$k$")
- How do we "practice" machine learning? What are the principles of good model-building?\
****

## Outline
1. Feature Engineering
2. The Bias-Variance Trade-Off
3. Cross Validation

# 1. Feature Engineering

## Feature Spaces
- The data that we get is just raw material: We have to deliberately create a **feature space** that allows models to be expressive and clever about using the data
- Already, we've seen various useful ways of processing the data: Log/IHS transformations, maxmin scaling, one-hot encoding
- When we create/transform new variables, we create new information that was previously unavailable, expanding the space of models that we can estimate

## Exercise
- Pick a dataset with a numeric outcome and some promising variables/features
- Discuss how you think those variables determine the target/outcome, in general
- We're then going to talk about strategies to build a feature space in general

In [57]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [58]:
df = pd.read_csv("data/metabric.csv")
df.head()

,Age at Diagnosis,Type of Breast Surgery,Cancer Type,Chemotherapy,Hormone Therapy,Lymph nodes examined positive,Mutation Count,Nottingham prognostic index,Overall Survival (Months),Overall Survival Status,Radio Therapy,TMB (nonsynonymous),Tumor Size,Tumor Stage
0,43.19,BREAST CONSERVING,Breast Cancer,NO,YES,0.0,2.0,4.020,84.633333,0:LIVING,YES,2.615035,10.0,1.0
1,48.87,MASTECTOMY,Breast Cancer,YES,YES,1.0,2.0,4.030,163.700000,1:DECEASED,NO,2.615035,15.0,2.0
2,47.68,MASTECTOMY,Breast Cancer,YES,YES,3.0,1.0,4.050,164.933333,0:LIVING,YES,1.307518,25.0,2.0
3,76.97,MASTECTOMY,Breast Cancer,YES,YES,8.0,2.0,6.080,41.366667,1:DECEASED,YES,2.615035,40.0,2.0
4,78.77,MASTECTOMY,Breast Cancer,NO,YES,0.0,4.0,4.062,7.800000,1:DECEASED,YES,5.230071,31.0,4.0


In [59]:
df.columns

Index(['Age at Diagnosis', 'Type of Breast Surgery', 'Cancer Type',
       'Chemotherapy', 'Hormone Therapy', 'Lymph nodes examined positive',
       'Mutation Count', 'Nottingham prognostic index',
       'Overall Survival (Months)', 'Overall Survival Status', 'Radio Therapy',
       'TMB (nonsynonymous)', 'Tumor Size', 'Tumor Stage'],
      dtype='object')

In [60]:
vars = ['Age at Diagnosis', 'Type of Breast Surgery', 'Cancer Type',
       'Chemotherapy', 'Hormone Therapy', 'Lymph nodes examined positive',
       'Mutation Count', 'Nottingham prognostic index',
       'Overall Survival (Months)', 'Overall Survival Status', 'Radio Therapy',
       'TMB (nonsynonymous)', 'Tumor Size', 'Tumor Stage']
vars_num = ['Age at Diagnosis', 'Tumor Size', 'Mutation Count','TMB (nonsynonymous)','Nottingham prognostic index']
X_num = df.loc[:, vars_num]
stage_ohc = pd.get_dummies(df["Tumor Stage"], dtype = int, drop_first = True)
chemo_ohc = pd.get_dummies(df["Chemotherapy"], dtype = int, drop_first = True)

In [61]:
X_num.head()

,Age at Diagnosis,Tumor Size,Mutation Count,TMB (nonsynonymous),Nottingham prognostic index
0,43.19,10.0,2.0,2.615035,4.020
1,48.87,15.0,2.0,2.615035,4.030
2,47.68,25.0,1.0,1.307518,4.050
3,76.97,40.0,2.0,2.615035,6.080
4,78.77,31.0,4.0,5.230071,4.062


In [62]:
X = pd.concat([X_num, chemo_ohc], axis = 1)
X.head()

,Age at Diagnosis,Tumor Size,Mutation Count,TMB (nonsynonymous),Nottingham prognostic index,YES
0,43.19,10.0,2.0,2.615035,4.020,0
1,48.87,15.0,2.0,2.615035,4.030,1
2,47.68,25.0,1.0,1.307518,4.050,1
3,76.97,40.0,2.0,2.615035,6.080,1
4,78.77,31.0,4.0,5.230071,4.062,0


In [63]:
from sklearn.linear_model import LinearRegression
y = df['Overall Survival (Months)']
model = LinearRegression()
model = model.fit(X,y)

model.coef_

array([ -1.40105318,  -0.56330518,  -7.96169211,   7.05273172,
       -11.62816492, -29.90857962])

In [64]:
from sklearn.linear_model import LinearRegression
y = df['Overall Survival (Months)']
model = LinearRegression()
model = model.fit(X_poly,y)

model.coef_

array([ 3.60222163e+00, -4.36510364e-01,  1.77006531e+02, -1.32086302e+02,
        3.15587766e+00, -3.54670305e+01, -4.74239015e-02,  1.47891950e-03,
       -3.35622839e+00,  2.55829912e+00,  2.07805143e-01, -1.79695646e-02,
        8.74607321e-03,  2.57272557e+00, -1.95646885e+00, -2.91442030e-01,
        4.29562015e-01, -2.40034157e+01,  3.99003975e+01, -7.19787593e+00,
       -7.08505477e+01, -1.64966262e+01,  5.16087643e+00,  5.26057343e+01,
       -2.28744457e+00,  1.00457518e+01, -3.54670305e+01])

In [65]:
X.columns

Index(['Age at Diagnosis', 'Tumor Size', 'Mutation Count',
       'TMB (nonsynonymous)', 'Nottingham prognostic index', 'YES'],
      dtype='object')

## We've already seen...
- Our most useful transformation is probably the natural logarithm (or inverse hyperbolic since when there are zeros or negative values): Converts variables with long tails into more bell-shaped ones
- One Hot Encoding: Transform one categorical/qualitative variable with $K$ labels into $K$ 0/1 variables that jointly encode the original label (remember to drop one label to avoid the dummy variable trap)

## Normalization/Standardization
- Especially with neighbor-based models and neural networks, normalizing the variables is important for computational stability
- Maxmin: 
$$
u_i = \frac{x_i - \min(x)}{\max(x)-\min(x)}
$$
- $z$-score: If all the variables are roughly bell-shaped (or are, after a log/ihs transformation), we can center them around 0 and give them a standard deviation of 1,
$$
z_i = \frac{x_i - \bar{x} }{s(x)}
$$
- Robust scaling: If there are outliers that make $z$-scaling unreliable,
$$
r_i = \frac{x_i - \text{median}(x)}{IQR(x)}
$$

## Missing Value Dummies
- For regression models, the following approach works very well:

1. Create a missing value dummy, `df['var_na'] = df['var'].isna()`
2. Replace missings with zero, `df['var_0'] = df['var'].nafill(0)`
3. Including both `var_na` and `var_0` in any regression

Why does this work?

$$
... + b_{\text{var}_{NA}} \text{var}_{NA} + b_{\text{var}_0} \text{var}_0+... = \begin{cases}
\beta_{\text{var}_0} \text{var}_0, & \text{ data available }\\
\beta_{\text{var}_{NA}}, & \text{ data missing }
\end{cases}
$$

- So the model smoothly switches back and forth between a linear model when data are available, and a missing data dummy when it's not available
- **Only do this with models based on linear combinations of variables**

## Polynomial Expansion
- In math, there's a fundamental concept (Stone-Weierstrass Theorem), that can be summarized like this: Any continuous function $\mu(x)$ can be approximated arbitrarily well by polynomial functions of the form
\begin{alignat*}{2}
m(x,b) &=& \underbrace{b_0}_{\text{0-order, constant, scalar}} + \underbrace{\sum_{\ell = 1}^L b_\ell x_\ell}_{\text{First-order, linear, vector}} \\
&& + \underbrace{\sum_{\ell =1}^L \sum_{j=1}^L b_{\ell j} x_\ell x_j}_{\text{Second-order, quadratic, matrix}} + \underbrace{\sum_{\ell =1}^L \sum_{j=1}^L \sum_{p=1}^L b_{\ell j p} x_\ell x_j x_p}_{\text{Third-order, quartic, tensor}} + \underbrace{...}_{\text{Higher order terms}} ,
\end{alignat*}
in the sense that $\max_{x} \min_{b} | \mu(x) - m(x,b) |$ can be made as small as desired by adding more higher order terms and optimizing over $b$
- This motivates our interest in **linear models**: We can approximate any **known** function $\mu(x)$ arbitrarily well with a model $m(x,b)$ where the $b$ coefficients are multiplicative with the data

## Polynomial Expansion in Scikit

1. **Import Expander**: `from sklearn.preprocessing import PolynomialFeatures`
2. **Create Expander**: `expander = PolynomialFeatures(degree=2,include_bias=False) # Create the expander`
3. **Fit Expander**: `X_poly = expander.fit_transform(X) `
4. **Store variable names**: `poly_manes = expander.get_feature_names_out() `


In [66]:
from sklearn.preprocessing import PolynomialFeatures
expander = PolynomialFeatures(degree=2,include_bias=False) # Create the expander
X_poly = expander.fit_transform(X)
X_poly

array([[43.19    , 10.      ,  2.      , ..., 16.1604  ,  0.      ,
         0.      ],
       [48.87    , 15.      ,  2.      , ..., 16.2409  ,  4.03    ,
         1.      ],
       [47.68    , 25.      ,  1.      , ..., 16.4025  ,  4.05    ,
         1.      ],
       ...,
       [52.84    , 20.      ,  5.      , ..., 25.4016  ,  5.04    ,
         1.      ],
       [48.59    , 30.      ,  6.      , ..., 25.6036  ,  5.06    ,
         1.      ],
       [63.2     , 22.      ,  3.      , ...,  9.265936,  0.      ,
         0.      ]], shape=(1343, 27))

In [67]:
expander.get_feature_names_out()

array(['Age at Diagnosis', 'Tumor Size', 'Mutation Count',
       'TMB (nonsynonymous)', 'Nottingham prognostic index', 'YES',
       'Age at Diagnosis^2', 'Age at Diagnosis Tumor Size',
       'Age at Diagnosis Mutation Count',
       'Age at Diagnosis TMB (nonsynonymous)',
       'Age at Diagnosis Nottingham prognostic index',
       'Age at Diagnosis YES', 'Tumor Size^2',
       'Tumor Size Mutation Count', 'Tumor Size TMB (nonsynonymous)',
       'Tumor Size Nottingham prognostic index', 'Tumor Size YES',
       'Mutation Count^2', 'Mutation Count TMB (nonsynonymous)',
       'Mutation Count Nottingham prognostic index', 'Mutation Count YES',
       'TMB (nonsynonymous)^2',
       'TMB (nonsynonymous) Nottingham prognostic index',
       'TMB (nonsynonymous) YES', 'Nottingham prognostic index^2',
       'Nottingham prognostic index YES', 'YES^2'], dtype=object)

## Interaction Terms
- Among the terms of the polynomial expansion we are most interested in, are terms of the type
$$
b_{jk} x_{j}x_{k}, \quad j \neq k
$$
or
$$
b_{jk\ell} x_{j}x_{k}x_{\ell}, \quad j \neq k \neq \ell
$$
which capture the joint non-linear impact of changes in $x_j$ and $x_k$ or $x_\ell$ together. 
- For example, the difference in price between a BMW with a sunroof versus without might be larger than the difference in price between a Honda with a sunfroof versus without. The BMW might exhibit more "luxury bang-for-buck" effects than more budget-conscious vehicles.
- Use `interaction_only=True` in Scikit's polynomial expander to get only the interactions, if you want to avoid high order powers of the variables alone

## Temporal Features/Controls (Optional)
- Some features/controls are periodic or recurrent, and a linear model seems like a bad fit: Think about hours of the day
- In cases like this, we can do two things

1. Bin the time variable into a categorical and one-hot encode (e.g. each hour of the day gets its own coefficient in the regression)
2. Use a Fourier Series:
$$
m(t,b) = b_0 + \sum_{\ell=1}^L \left[ b_{\ell,\cos} \cos \left(2 \pi \ell \dfrac{ t}{T} \right) + b_{\ell,\sin} \sin \left( 2 \pi \ell \dfrac{t}{T} \right) \right],
$$
where $T$ is the period length ($T=24$ for hours, $T=7$ for day-of-week, $T=365$ for day-of-year, etc.)

The Fourier Series is still a linear model in the $b$ coefficients, just like the polynomial family, but it's built from sines and cosines, so it behaves periodically and continuously as $t/T$ ranges over its possible values. 

## Fourier Series Basis Functions (Optional)
- These are like polynomial families $1, x, x^2, x^3,...$, but for variation over time:
<img src="./src/fourier.png" alt="Illustration of bias and variance." width="100%" height="auto">

## Time Series: Temporal Target/Outcome (Optional)
- Imagine your target variable is denominated in time, so it's the $y_{it}$ rather than $x_{it}$ that is temporal
- Instead of studying $y_{it}$ directly, take the **first difference**
$$
z_t = y_t - y_{t-1}
$$
or compute the **log growth** or **log return** (particularly in fenance)
$$
r_t = \log \left( \frac{ y_t }{ y_{t-1} } \right)
$$
- In regression, we often include **lags** of the variable to account for its "momentum" or "trajectory" over time, like
$$
\hat{z}_t = b_0 + b_1 z_{t-1} + b_2 z_{t-2} + ... + b_K z_{t-K}
$$
So we model the variable's evolution over time based on recent changes in its values, rather than levels

## Exercise
- Engineer a feature space for your data
- Do a train-test split
- Train a simple linear model on the training data
- Train a complex linear model on the training data
- Which does better (lower MSE) on the test set?

# 2. The Bias-Variance Trade-Off

## Machine Learning
- Many fields are driven by a handful of core ideas
- For machine learning/data science, one of these core ideas is called the **bias-variance trade-off**
- This is a mathematical concept and we'll briskly cover the quantitative argument behind it, and then focus on the intuition it provides
- This discussion is mathematical, to show how you discover and think about an idea like this
- This lesson is essentially the "general theory of machine learning"

## Expectation
- One of our favorite quantities is the sample mean of a variable $X$,
$$
m(X) = \sum_{i=1}^N \frac{1}{n} x_i
$$
- We want to imagine a similar concept: What value of $X$ is most likely, before we've observed the data?
- We are going to replace $1/n$ with the abstract probability that $X=x$ occurs, $p(x)$, and sum over possible values of $X$
- The **expectation of a random variable $X$** is
$$
\mathbb{E}_X[X] = \sum_{\text{All } x \text{ such that }p(x)>0} p(x) x
$$

## Expectation
- Another way to understand this: Imagine computing $m(X)$ by first computing the histogram, and weighting each $x_i$ by the height of the histogram
- Another way to understand this: It answers the question, "Before we actually gather any data, what do you expect the sample mean to be?"
- As data scientists we love this: It's like we're thinking about the model before we actually gather data or do any analysis
- Key observation: $\mathbb{E}_X[X]$ is just a number, like 7.46. So $\mathbb{E}_X[ \mathbb{E}_X[X] ] = \mathbb{E}_X[X]$ (idempotent operator)

## Expectation: Example
- What is the expected value of rolling a single die?
- What is the expected value of rolling two dice and summing the results?

## Bias and Model Variance
- There are two properties of an estimate that it's important to keep in mind:

1. **Bias**: If the truth is $T$ and we predicted $\hat{T}$, the bias is equal to $\mathbb{E}_{\hat{T}}[\hat{T} - T]$, the expected difference between the true value and out predicted value
2. **Variance**: The variability of our prediction is $\mathbb{E}_{\hat{T}}[ (\hat{T}-T)^2 ]$

We obviously don't like bias: If it is large, we make lots of mistakes in expectation. But it turns out that we also don't like variance: If the variance is large, our predictions are typically unreliable.

## Bias and Variance: Classic Picture

<img src="./src/bvt.png" alt="Illustration of bias and variance." width="65%" height="auto">


## Bias-Variance Trade-Off
- It turns out that bias and variance are the core considerations of how we build models
- There's a bunch of features/covariates $x_1, x_2, ..., x_L$, which jointly determine the expected value of $y$, which is $\mathbb{E}_X[y|X] = \mu(x_1, x_2, ..., x_L)$
- However, there's an additive shock, $\varepsilon$, with mean zero, so we never observe $\mu$ directly:
$$
y(x_1, x_2, ..., x_L) = \mu(x_1, x_2, ..., x_L) + \varepsilon
$$
- We assume the expected value of $\varepsilon$ is $\mathbb{E}[\varepsilon] = 0$ and its variance is $\mathbb{E}[\varepsilon^2] = \sigma^2$

## Bias-Variance Trade-Off
- Given this set-up, what is machine learning?
- What we're doing is guessing $\mu(x_1, x_2, ..., x_L)$ with a model $m(x_1, x_2, ..., x_L; b)$, and then choosing $b$ to solve this:
$$
\min_{b} \frac{1}{n} \sum_{i=1}^n (\mu(x_i) + \varepsilon_i - m(x_i,b))^2
$$
- Our model, $m(x,b)$ is just a model: $k$-NN, linear, whatever we want.

## Expected Loss
- **Before** we gather the data, what do we **expect**ed to happen with our loss function (MSE)?
\begin{alignat*}{2}
\mathbb{E}_{X,\varepsilon}\left[ \frac{1}{n} \sum_{i=1}^n (\mu(x_i) + \varepsilon_i - m(x_i,b))^2 \right] 
\end{alignat*}
- Let's work out how to simplify this There's essentially two tricks: FOIL and simplify, then add/subtract and FOIL again, using the expected value to erase a surprising number of terms

## Bias-Variance Trade-Off
- So our MSE breaks into three distinct pieces:
\begin{alignat*}{2}
\mathbb{E}_{X,\varepsilon}\left[ \text{MSE}(b) \right] &=& \underbrace{\mathbb{E}_{X,\varepsilon}\left[ \frac{1}{n} \sum_{i=1}^n  (\mu(x_i) - \mathbb{E}_{X}[m(X,b)])^2 \right]}_{\text{Predictor Bias}} + \underbrace{\mathbb{E}_{X,\varepsilon}\left[ \frac{1}{n} \sum_{i=1}^n (\mathbb{E}_{X}[m(X,b)]- m(x_i,b))^2 \right]}_{\text{Model Variance}}\\
&& + \underbrace{\sigma^2}_{\text{Irreducible Error}}
\end{alignat*}
and the take-away intuition from this equation is that
$$
\text{Expected Loss} = \text{Bias-Squared} + \text{Model Variance} + \text{Irreducible Error}
$$

## Bias-Variance Trade-Off

- So our expected loss decomposes into three pieces:
    1. **Predictor Bias**: In expectation, how far do we expect our model $m$ to be from the true $\mu$?
    2. **Model Variance**: Regardless of the truth, how variable is our model in expectation?
    3. **Irreducible Error**: How much inherent noise is there in the environment?

## BVT for $k$-NN
- For the $k$-neighbors regressor model, there's an explicit formula:
$$
\mathbb{E}_{X,y}[(y-m(x))^2|X=x] = \underbrace{\left( \mu(x) - \frac{1}{k}\sum_{\text{$i$ in Neighbors}(x)}^K y_i \right)^2}_{\text{Bias squared}} + \underbrace{\frac{\sigma^2}{k}}_{\text{Variance}} + \underbrace{\sigma^2}_{\text{Irreducible Error}}
$$
- As $k$ increases, the model variance decreases: You are averaging over more and more neighbors, so the predictor variance shrinks
- As $k$ increases, the bias squared increases: You are averaging over cases that are further and further from $x$, so in expectation, they are becoming less relevant
- This is the BVT in practice: A very low $k$ and a very high $k$ are typically both not the solution, and the optimal $k$ depends on the prediction environment in question

## Optimal Model Complexity

<img src="./src/model_complexity.png" alt="Illustration of model complexity trade-off." width="75%" height="auto">

## Over-fitting, Under-fitting
- Excessively complex models are associated with high variance but low bias, leading to high expected loss: This is called **over-fitting**
- Excessively simplistic models are associated with low variance but high bias, leading to high expected loss: This is called **under-fitting**
- We want to balance **parsimony** with **verisimilitude**, and avoid both under- and over-fitting
- This is what the train-test split is about: Using data to judge when the model is transitioning from simple and robust to overly complicted and unreliable
- This requires judgment and appropriate use of model selection tools
- This is a fundamental concept that we'll talk about for the rest of class, and the kind of idea you should commit to your data science core archive of ideas

# 3. Cross Validation

## Model Selection and Validation
- Ok, we can build massive and complex feature spaces...
- ... but we know this will typically lead to overfitting and needlessly complex models that aren't BVT-optimal
- How do we estimate expected performance, so we can address concerns about the bias-variance trade-off?

## Exercise
- For your data, pick a model selection/hyperparameter choice about which you are uncertain
- For example, "How many powers of $x$ should I include?", "How many lags of the time series should I use?", "Should I include this variable or not?"

## $K$-Fold Cross Validation
- This is a kind of "$K$-way train/test split":

0. Fix the model (the variables, the neighbors, how many polynomial features, etc.)
1. Partition the data into $K$ equally-sized sets (the folds)
2. For each $k=1,...,K$, estimate your model's parameters on the complementary chunks, and test performance on the the $k$-th chunk
3. Save the model's performance for each chunk (e.g. accuracy, MSE)

This provides $K$ estimates of the model's performance, estimating the distribution function of the model's performance


<img src="./src/crossvalidation.png" alt="Illustration of $k$-Fold Cross Validation." width="80%" height="auto">


## Scikit Implementation: Mostly Automatic

``` python 
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error

[...]

kfold = KFold(n_splits=10, shuffle=True, random_state=100) # Create folds

scores = cross_val_score( # Conduct kfcv:
    model,X,y, # Model and data
    cv=kfold, # Folds
    scoring=mean_squared_error # Loss function
)

print("Fold scores:", scores)
print("Mean score:", np.mean(scores))
print("Median score:", np.median(scores))
print("Std dev:", np.std(scores))
```

## Scikit Implementation: More Control

``` python 
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

[...]

kf = KFold(n_splits=10, shuffle=True, random_state=42)

scores = []
for train_idx, test_idx in kf.split(X):
    X_train, X_test = X[train_idx], X[val_idx]
    y_train, y_test = y[train_idx], y[val_idx]

    model.fit(X_train, y_train)
    y_hat = model.predict(X_test)
    score = mean_squared_error(y_test, y_hat)
    scores.append(score)

print("Fold scores:", scores)
print("Mean score:", np.mean(scores))
print("Median score:", np.median(scores))
print("Std dev:", np.std(scores))

```

## Exercise
- For your model selection question, use $k$-FCV to estimate the model's performance for a variety of plausible alternative, and pick the alternative with the best median or average performance.

## Conclusion
- This was a lot! 
- Feature engineering is essential to build realistic, expressive models
- But we can go too far, and end up over-fitting, or not far enough, and under-fit: this is the essence of the BVT
- Cross validation allows us to quantify whether we're doing a good job or not
- These are fundamental topics in the field of ML/DS
- The connections with probability and simulation can be further developed, but getting some experience with the ideas first is a good idea
- While we're here: The point of deep learning is to use a neural network to automatically do feature engineering for us (representation learning)